# Imports

In [1]:
import os
from typing import Dict, List, Tuple

import warnings

from adodbapi.process_connect_string import process
from pyarrow.dataset import dataset

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics, DatasetWin, calculate_metrics_from_df

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 60                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"
DEPRESJON_PREFIX = "manual_depresjon"
PSYKOSE_PREFIX = "manual_psykose"
HYPERAKTIV_PREFIX = "manual_hyperaktiv"
MAIN_RESULTS_DIR = "results"
DAY_WINDOWS_DIR = "day_windows"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[List[pd.DataFrame]]) -> List[List[pd.DataFrame]]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - Format "timestamp" to YYYY-MM-DD HH:MM:SS
    - Drop redundant "date" column
    - Convert "activity" to float32
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    for patient in data:
        for df in patient:
            # Convert and enforce the desired timestamp format
            df["timestamp"] = pd.to_datetime(df["timestamp"], dayfirst=False)
            df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
            
            # Drop "date" column if it exists
            if "date" in df.columns:
                df.drop("date", axis=1, inplace=True)
            
            # Ensure "activity" column is float32
            df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame, freq: str = "min") -> pd.DataFrame:
    """
    Fill missing activity values by resampling based on given frequency.
    
    :param df: DataFrame with 'timestamp' and 'activity' columns.
    :param freq: Resampling frequency (default: minute).
    :return: DataFrame with missing values filled.
    """
    df = df.copy() # create copy to avoid side effects
  
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df.set_index("timestamp", inplace=True)

    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample(freq).mean()
    
    # recreate index and "timestamp" column
    df.reset_index(inplace=True)

    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())
    
    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 
    
    psd = scipy.signal.welch(
        x=activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 

    power_spectrum = scipy.signal.welch(
        activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[List[pd.DataFrame]], is_condition: bool = True, freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of lists of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [[fill_missing_activity(df) for df in patient] for patient in full_dfs]
    full_dfs = [[resample(df, freq=freq) for df in patient] for patient in full_dfs]
    night_dfs = [[get_day_part(df, part="night") for df in patient] for patient in full_dfs]
    day_dfs = [[get_day_part(df, part="day") for df in patient] for patient in full_dfs]

    datasets = {}
    
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs), ("day", day_dfs)]:
        features = []
        for patient in range(len(list_of_dfs)):
            for day in range(len(list_of_dfs[patient])):
                time_features = extract_time_features(list_of_dfs[patient][day])
                freq_features = extract_frequency_features(list_of_dfs[patient][day])
    
                merged_features = pd.merge(
                    time_features,
                    freq_features,
                    left_index=True,
                    right_index=True,
                    suffixes=("_time", "_freq")
                )
                merged_features['day'] = day + 1
                merged_features['patient_id'] = patient + 1
                if is_condition:
                    merged_features['class'] = 1
                else:
                    merged_features['class'] = 0
                features.append(merged_features)
    
        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets

## Hyperaktiv

In [7]:
path = os.path.join(PROCESSED_DATA_DIR, DAY_WINDOWS_DIR, "hyperaktiv")
dataset = DatasetWin(dirpath=path, sep=',')
condition = dataset.condition
control = dataset.control

In [8]:
condition[0][0]

,timestamp,activity
0,2009-02-23 16:00:00,0.0
1,2009-02-23 16:01:00,195.0
2,2009-02-23 16:02:00,240.0
3,2009-02-23 16:03:00,209.0
4,2009-02-23 16:04:00,202.0
...,...,...
1435,2009-02-24 15:55:00,195.0
1436,2009-02-24 15:56:00,80.0
1437,2009-02-24 15:57:00,104.0
1438,2009-02-24 15:58:00,83.0


In [9]:
condition_parts_dfs = extract_features_for_dataframes(condition, is_condition=True, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, is_condition=False, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    max_patient = condition_df['patient_id'].max()
    control_df['patient_id'] += max_patient # changing numeration of patients
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [10]:
entire_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness,day,patient_id,class
0,5.450000,215.399994,98.082039,85.000000,4445.753906,-1.023036,0.421944,0.653134,98.533337,95.837883,...,-1.102940,0.239344,0.399889,2.704821e+05,3.863297e+05,4.836528,0.915487,1,1,1
1,0.000000,116.266670,20.491024,4.550000,1133.641479,3.115266,1.988219,1.578676,31.050000,13.646970,...,0.414452,1.392186,1.345802,5.072696e+04,3.355082e+04,3.861883,0.358044,2,1,1
2,0.000000,82.599998,15.717948,3.916667,602.977356,2.412652,1.903432,1.500975,10.566666,11.066667,...,-0.534145,0.809441,0.922332,3.391282e+04,2.106232e+04,4.333512,0.533665,3,1,1
3,0.000000,403.383331,105.644882,12.183333,20564.833984,-0.578848,0.989674,1.304167,208.750007,88.181816,...,-0.892984,0.815363,0.946616,5.788349e+06,3.397530e+06,4.330836,0.577943,4,1,1
4,11.766666,138.433334,50.897438,38.266666,1330.755493,0.478558,1.065593,0.688608,48.183331,46.496967,...,-0.141829,0.944444,0.863809,1.029162e+05,8.174428e+04,4.422207,0.564267,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,119.316666,510.299988,260.858948,229.300003,17234.105469,-0.353035,0.891895,0.483512,140.633331,251.050003,...,-0.154728,0.842434,0.659194,1.692111e+06,2.098906e+06,4.643049,0.725992,2,85,0
543,79.416664,419.983337,215.976913,216.699997,9484.760742,-0.518492,0.416418,0.433236,142.733337,209.845459,...,-1.159992,0.309771,0.696326,1.098544e+06,9.513142e+05,4.569425,0.672638,3,85,0
544,26.616667,1055.616699,360.837189,323.516663,77397.132812,1.050322,1.152404,0.740747,256.833328,328.059082,...,-0.884483,0.799677,1.081417,1.201404e+07,5.903548e+06,4.095445,0.379739,4,85,0
545,0.000000,622.066650,160.816650,106.333336,39549.644531,0.284912,1.184161,1.188117,165.816674,133.504547,...,1.154687,1.429178,0.999763,1.225944e+06,1.117923e+06,4.296974,0.411765,5,85,0


In [11]:
datasets['day']

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness,day,patient_id,class
0,5.450000,215.399994,98.082039,85.000000,4445.753906,-1.023036,0.421944,0.653134,98.533337,95.837883,...,-1.102940,0.239344,0.399889,2.704821e+05,3.863297e+05,4.836528,0.915487,1,1,1
1,0.000000,116.266670,20.491024,4.550000,1133.641479,3.115266,1.988219,1.578676,31.050000,13.646970,...,0.414452,1.392186,1.345802,5.072696e+04,3.355082e+04,3.861883,0.358044,2,1,1
2,0.000000,82.599998,15.717948,3.916667,602.977356,2.412652,1.903432,1.500975,10.566666,11.066667,...,-0.534145,0.809441,0.922332,3.391282e+04,2.106232e+04,4.333512,0.533665,3,1,1
3,0.000000,403.383331,105.644882,12.183333,20564.833984,-0.578848,0.989674,1.304167,208.750007,88.181816,...,-0.892984,0.815363,0.946616,5.788349e+06,3.397530e+06,4.330836,0.577943,4,1,1
4,11.766666,138.433334,50.897438,38.266666,1330.755493,0.478558,1.065593,0.688608,48.183331,46.496967,...,-0.141829,0.944444,0.863809,1.029162e+05,8.174428e+04,4.422207,0.564267,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,119.316666,510.299988,260.858948,229.300003,17234.105469,-0.353035,0.891895,0.483512,140.633331,251.050003,...,-0.154728,0.842434,0.659194,1.692111e+06,2.098906e+06,4.643049,0.725992,2,85,0
543,79.416664,419.983337,215.976913,216.699997,9484.760742,-0.518492,0.416418,0.433236,142.733337,209.845459,...,-1.159992,0.309771,0.696326,1.098544e+06,9.513142e+05,4.569425,0.672638,3,85,0
544,26.616667,1055.616699,360.837189,323.516663,77397.132812,1.050322,1.152404,0.740747,256.833328,328.059082,...,-0.884483,0.799677,1.081417,1.201404e+07,5.903548e+06,4.095445,0.379739,4,85,0
545,0.000000,622.066650,160.816650,106.333336,39549.644531,0.284912,1.184161,1.188117,165.816674,133.504547,...,1.154687,1.429178,0.999763,1.225944e+06,1.117923e+06,4.296974,0.411765,5,85,0


In [38]:
datasets['day'].isnull().sum()

minimum_time           0
maximum_time           0
mean_time              0
median_time            0
variance_time          0
kurtosis_time          1
skewness_time          1
coeff_of_var_time      1
iqr_time               0
trimmed_mean_time      0
entropy_time           1
proportion_of_zeros    0
minimum_freq           0
maximum_freq           0
mean_freq              0
median_freq            0
variance_freq          0
kurtosis_freq          1
skewness_freq          1
coeff_of_var_freq      1
iqr_freq               0
trimmed_mean_freq      0
entropy_freq           1
spectral_flatness      1
day                    0
patient_id             0
class                  0
dtype: int64

In [39]:
datasets['night'].isnull().sum()

minimum_time           0
maximum_time           0
mean_time              0
median_time            0
variance_time          0
kurtosis_time          2
skewness_time          2
coeff_of_var_time      2
iqr_time               0
trimmed_mean_time      0
entropy_time           2
proportion_of_zeros    0
minimum_freq           0
maximum_freq           0
mean_freq              0
median_freq            0
variance_freq          0
kurtosis_freq          2
skewness_freq          2
coeff_of_var_freq      2
iqr_freq               0
trimmed_mean_freq      0
entropy_freq           2
spectral_flatness      2
day                    0
patient_id             0
class                  0
dtype: int64

In [12]:
# save manual features
# os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
# for part, df in datasets.items():
#     filename = f"{HYPERAKTIV_PREFIX}_window_{part}.csv"
#     filepath = os.path.join(PROCESSED_DATA_DIR, filename)
#     df.to_csv(filepath, index=False, header=True)

# Classification

## Classifiers, parameters, constants

In [14]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Hyperaktiv Classification

In [15]:
dataset = HYPERAKTIV_PREFIX

In [ ]:
# create dictionary with data split for night/day/all
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_window_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0)

y = datasets['day']['class']

In [ ]:
# X = pd.concat([datasets['full_24h'].iloc[:25, :], datasets['full_24h'].iloc[-25:, :]])
# X = X.reset_index(drop=True)

# y = pd.concat([y[:25], y[-25:]])
# y = y.reset_index(drop=True)


In [35]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "hyperactiv")
predictions_directory = os.path.join('.', MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "hyperactiv", "predictions")
os.makedirs(results_directory, exist_ok=True)
os.makedirs(predictions_directory, exist_ok=True)
predictions = pd.DataFrame(columns=['fold', 'classifier', 'predicted_class', 'actual_class', 'patient_id'])

for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    X = datasets[part]
    info = X.iloc[:, -3:]
    group_kfold = GroupKFold(n_splits=3)
    fold_num = 0
    for train_idx, test_idx in group_kfold.split(X, y, groups=X['patient_id']):
        fold_num += 1 
        X = X.iloc[:, :-3]
        print("fold: ", fold_num)
        print(len(train_idx))
        print(len(test_idx))
        
        np.random.shuffle(train_idx)
        np.random.shuffle(test_idx)

        print(test_idx)
    
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.005)
        X_train, X_test = standardize(X_train, X_test)

        test_scores = []
        for clf_type in ["LR", "SVM", "RF"]: 
            print(f"    {clf_type}")
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred = clf.predict(X_test)
            
            print(y_pred)
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)

            # Save individual fold metrics
            pd.DataFrame.from_records(test_scores).to_csv(
                os.path.join(results_directory, f"test_scores_{part}_fold_{clf_type}"),
                index=False
            )

            for idx, pred in enumerate(y_pred):
                y_test_val = y_test[test_idx[idx]]

                new_row = {
                    'fold': fold_num,
                    'classifier': clf_type,
                    'predicted_class': pred, 
                    'actual_class': y_test_val, 
                    'patient_id': info.loc[test_idx[idx], 'patient_id'] 
                }

                predictions = pd.concat([predictions, pd.DataFrame([new_row])], ignore_index=True)

        # Compute and save final scores for the fold
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                          columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(
            os.path.join(results_directory, f"final_scores_{part}_fold"),
            index=False
        )

        for metric, (mean, stddev) in final_scores.items():
            print(f"      {metric}: {mean:.4f} +- {stddev:.4f}")
        print()
    
    predictions.to_csv(
    os.path.join(predictions_directory, f"predictions_{part}.csv"),
    index=False
    )


PART: full_24h
fold:  1
365
182
[395 182  42  97 307 134 343 175 267 268 498 224 185   6 311 302  50 195
  98 300 266 371 348 370 374  48 415 308 170 342 132 345 196 323 176 180
 414 401 298   7  95 192 194   8 352  40  46 183 439 269 133 530 190 129
 131   5 303 191  27 373 346 438 161  31 165 172 243 412 347 397 241 531
 188 496 309 325   0 354 413 222 441 306 324 128 100 226 246 372  96   9
  11  28  29 244 321   3 529 162 351 186 326 528  47 322 495 297  43 177
 444 187 179  67 301  41 169  84 411  94 499 171  45 494  66 270 375 245
  69  64 399 178 130 189 193 184 242  82 167 344 174 299 497  65 527 181
   4 398 532 264 443 409  99 327 305 164 350 410 166  10 225 353 396   2
  44 349  30 265  49 440  32  68 304 168  39 310 173 400  51 442  83   1
 223 163]
    LR


KeyboardInterrupt: 

### Voting

In [37]:
for part in ["full_24h"]:
    predictions = pd.read_csv(os.path.join(predictions_directory, f"predictions_{part}.csv"))
    print(predictions)

    grouped = predictions.groupby(['patient_id', 'classifier'])

    most_common_class = (
        grouped['predicted_class']
        .apply(lambda x: x.mode()[0]) 
        .reset_index(name='final_predicted_class')
    )

    final_results = pd.merge(
        most_common_class,
        predictions[['patient_id', 'actual_class']].drop_duplicates(),
        on='patient_id'
    )

    print(final_results)

    final_results.to_csv(
        os.path.join(predictions_directory, f"final_predictions_{part}.csv"),
        index=False
    )

    voting_metrics = (
    final_results.groupby('classifier')
    .apply(lambda group: pd.Series(
        calculate_metrics_from_df(group['actual_class'], group['final_predicted_class'])
    ))
    .reset_index()
    )

    print(voting_metrics)
    
    voting_metrics.to_csv(
    os.path.join(predictions_directory, f"voting_scores_{part}.csv"),
    index=False
    )

     fold classifier  predicted_class  actual_class  patient_id
0       1         LR                1             0           4
1       1         LR                1             1          82
2       1         LR                1             0          85
3       1         LR                1             1          82
4       1         LR                0             0          84
..    ...        ...              ...           ...         ...
145     2         RF                0             1           1
146     2         RF                0             1           1
147     2         RF                0             1           3
148     2         RF                1             1           3
149     2         RF                1             1           2

[150 rows x 5 columns]
    patient_id classifier  final_predicted_class  actual_class
0            1         LR                      0             1
1            1         RF                      0             1
2            1     

## Depresjon

In [306]:
path = os.path.join(PROCESSED_DATA_DIR, DAY_WINDOWS_DIR, "depresjon")
dataset = DatasetWin(dirpath=path, sep=',')
condition = dataset.condition
control = dataset.control

In [307]:
condition[0][0]

,timestamp,activity
0,2003-05-07 12:00:00,0.0
1,2003-05-07 12:01:00,143.0
2,2003-05-07 12:02:00,0.0
3,2003-05-07 12:03:00,20.0
4,2003-05-07 12:04:00,166.0
...,...,...
1435,2003-05-08 11:55:00,259.0
1436,2003-05-08 11:56:00,190.0
1437,2003-05-08 11:57:00,306.0
1438,2003-05-08 11:58:00,91.0


In [308]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [310]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [313]:
datasets["full_24h"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness,day,patient_id,class
0,0.966667,395.649994,131.482635,105.699997,16655.847656,-0.920654,0.638463,0.960889,212.062506,120.149170,...,4.867227,2.483317,2.018346,2.211316e+05,5.500495e+05,3.240957,0.281053,1,1,1
1,0.916667,569.866638,172.020142,84.541672,33375.023438,-0.561575,0.833602,1.039656,304.258337,150.076660,...,4.082882,2.325901,1.870382,1.386454e+06,1.227904e+06,3.344774,0.230570,2,1,1
2,3.500000,505.100006,150.905563,80.291664,27618.832031,-0.605859,0.969403,1.078092,218.149999,132.926666,...,4.611042,2.451845,2.050603,5.640596e+05,6.595294e+05,3.154835,0.232096,3,1,1
3,3.950000,362.500000,121.906250,98.341667,12383.744141,-0.782771,0.595389,0.893631,188.479172,111.311668,...,2.667433,2.058466,1.848006,3.135061e+05,5.504776e+05,3.325392,0.262556,4,1,1
4,0.000000,857.316650,144.522919,0.133333,48419.500000,2.665590,1.721625,1.490499,212.262501,106.300827,...,4.504607,2.411578,2.078014,8.499194e+05,1.027154e+06,3.068435,0.186906,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,13.066667,851.250000,315.575684,280.450012,64837.890625,-0.624075,0.691747,0.789895,247.808332,297.199982,...,3.436387,1.933112,1.188187,3.639221e+06,2.119670e+06,4.132731,0.347506,16,32,1
1059,4.466667,635.766663,253.211121,237.600006,35579.738281,-0.868062,0.345004,0.729251,300.741673,241.473343,...,0.983052,1.418376,1.329144,4.870405e+06,2.246511e+06,3.808833,0.224110,17,32,1
1060,1.366667,465.066681,187.743759,215.050003,24700.861328,-1.422475,0.099830,0.819500,295.824997,180.345016,...,3.644140,2.224552,1.707360,1.365883e+06,1.374905e+06,3.576127,0.334359,18,32,1
1061,5.100000,1052.849976,282.377777,243.300003,72021.414062,0.955541,1.148813,0.930376,343.291672,250.560837,...,4.556210,2.346643,1.488078,3.786963e+06,3.001285e+06,3.887931,0.446845,19,32,1


## Depresjon classification

In [314]:
dataset = DEPRESJON_PREFIX

In [317]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0)

In [ ]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "depresjon")

# for part in ["full_24h", "night", "day"]:
for part in ["night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{part}_{clf_type}"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{part}_{clf_type}"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

## Psykose

In [ ]:
dataset = Dataset(dirpath=os.path.join("data", "psykose"))
condition = dataset.condition
control = dataset.control

In [ ]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [ ]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [ ]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)

## Psykose classification

In [ ]:
dataset = PSYKOSE_PREFIX
y_filename = "psykose_y.csv"

In [ ]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [ ]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "psykose")

# for part in ["full_24h", "night", "day"]:
for part in ["night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{part}_{clf_type}"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{part}_{clf_type}"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()